In [1]:
from alntk.alignment import Alignment
from alntk.plotting import default_plot_style

import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_folder = '../data/'
color_cycle = default_plot_style()

# Try to do it on alignments...

In [5]:
aln_halabi = Alignment()
aln_halabi.import_from_fasta(data_folder + 'serprot_ref.faa')

In [7]:
aln_hmmsearch = Alignment()
aln_hmmsearch.import_from_fasta(data_folder + 'new_aln0.faa')

In [49]:
import numpy as np

In [60]:
nonzero_pos = np.where(np.sum(aln_halabi.get_seqs() != '-', axis=0) > 0)[0]

In [61]:
seqs_halabi = [''.join(e) for e in aln_halabi.get_seqs()[:, nonzero_pos]]
seqs_hmmsearch = [''.join(e) for e in aln_hmmsearch.get_seqs()]

In [62]:
df_halabi = pd.DataFrame({
    'descs': aln_halabi.get_descs(),
    'seqs': seqs_halabi
})

In [63]:
df_hmmsearch = pd.DataFrame({
    'descs': aln_hmmsearch.get_descs(),
    'seqs': seqs_hmmsearch
})

In [64]:
len(df_hmmsearch.seqs.iloc[1])

823

In [65]:
len(df_halabi.seqs.iloc[0])

823

In [74]:
set(df_halabi.seqs).intersection(df_hmmsearch.seqs)

set()

In [79]:
seqs_halabi[0].replace('-', '')

'IVGGYTCQENSVPYQVSLNSGYHFCGGSLINDQWVVSAAHCYKSRIQVRLGEHNINVLEGNEQFVNAAKIIKHPNFDRKTLNNDIMLIKLSSPVKLNARVATVALPSSCAPAGTQCLISGWGNTLSSGVNEPDLLQCLDAPLLPQADCEASYPGKITDNMVCVGFLEGGKDSCQGDSGGPVVCNGELQGIVSWGYGCALPDNPGVYTKVCNYVDWIQDTIAAN'

# Just BLAST and pair

In [6]:
df = pd.read_csv(data_folder + 'match_halabi/match_output_reduced.csv', sep='\t', header=None)
df.columns = ['halabi_desc', 'hmmsearch_desc', 'evalue']

In [17]:
df

,halabi_desc,hmmsearch_desc,evalue
0,gi|4139558|pdb|3TGI|E__vertebrate|warm|Rattus,sp|P00763|TRY2_RAT/3246,2.710000e-166
1,gi|4139558|pdb|3TGI|E__vertebrate|warm|Rattus,tr|A6IF27|A6IF27_RAT/3246,9.660000e-166
2,gi|4139558|pdb|3TGI|E__vertebrate|warm|Rattus,tr|A0A822IED2|A0A822IED2_9NEOB/3246,9.660000e-166
3,gi|4139558|pdb|3TGI|E__vertebrate|warm|Rattus,tr|W4VSR6|W4VSR6_RAT/2246,2.050000e-164
4,gi|4139558|pdb|3TGI|E__vertebrate|warm|Rattus,tr|A6IF30|A6IF30_RAT/2246,2.050000e-164
...,...,...,...
695394,gi|24660124__not,tr|I7I4B9|I7I4B9_SCYPA/30268,2.990000e-16
695395,gi|24660124__not,tr|A0A1I8M1T7|A0A1I8M1T7_MUSDO/28299,3.030000e-16
695396,gi|24660124__not,tr|A0A3G5BIH8|A0A3G5BIH8_DOLGE/3264,3.160000e-16
695397,gi|24660124__not,tr|Q8SYB5|Q8SYB5_DROME/26273,3.260000e-16


In [26]:
min_evalue_idx = df.groupby('halabi_desc')['evalue'].idxmin()

In [30]:
df_min = df.iloc[min_evalue_idx]

In [33]:
df_min['gi_code'] = df_min.halabi_desc.apply(lambda x: x.split('|')[1].split('_')[0])

/tmp/ipykernel_18453/4003957553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_min['gi_code'] = df_min.halabi_desc.apply(lambda x: x.split('|')[1].split('_')[0])


In [34]:
df_min['upkb_code'] = df_min.hmmsearch_desc.apply(lambda x: x.split('|')[1])

/tmp/ipykernel_18453/1165234776.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_min['upkb_code'] = df_min.hmmsearch_desc.apply(lambda x: x.split('|')[1])


In [39]:
df_min[['gi_code', 'upkb_code']].reset_index(drop=True).to_csv(data_folder + 'halabi_newaln_mapping.csv', index=False)